In [3]:
import pandas

In [4]:
import csv

Read in the mask data from the NYT github (note: this data is at the county level for the United States). 

In [5]:
mask = pandas.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv')

mask #view data



In [ ]:
Write the raw daw data to disc to upload to the github folder 

In [ ]:
mask.to_csv('raw_mask_data.csv')

We are interested in creating a composite 'mask score' for each county. 

Below is information on the raw mask data and how to create a composite mask score for each county. 

COUNTYFP: The county FIPS code.
NEVER: The estimated share of people in this county who would say never in response to the question “How often do you wear a mask in public when you expect to be within six feet of another person?”
RARELY: The estimated share of people in this county who would say rarely
SOMETIMES: The estimated share of people in this county who would say sometimes
FREQUENTLY: The estimated share of people in this county who would say frequently
ALWAYS: The estimated share of people in this county who would say always


NOTE: Calculate a Mask Score for each county = (%NEVER*0) + (%RARELY*1) + (%SOMETIMES*2) +
(%FREQUENTLY*3) + (%ALWAYS*4)

Source: (Maloney, 2020): https://www.medrxiv.org/content/10.1101/2020.10.03.20206326v2.full.pdf



Importantly, the mask data only has county fip codes, not the names of each county. My group will be merging our data at the county level using the name of the county, so I need to identify the corresponding county names for the fip codes. To do this, I need to read in a dataset that has the fip codes and the county names:

County Fip codes + name if county : https://github.com/kjhealy/fips-codes/blob/master/state_and_county_fips_master.csv

Area Type: county Geoid structure = state + county


We are interested in county level data in Washington State, California, and Oregon. I need to:

(a) Identify the corresponding county names of the County FIP codes (census data) to append to the the mask data
(b) Filter the data to only include counties in WA,CA, and OR
(c) Create a mask score for each county
(d) Change variable names to prepare the data to be merged with my other group members


County Fip codes:
    
Area Type: county 
Geoid structure: state + county

https://github.com/kjhealy/fips-codes/blob/master/state_and_county_fips_master.csv


In [20]:
fip = pandas.read_csv('https://raw.githubusercontent.com/kjhealy/fips-codes/master/state_and_county_fips_master.csv')

fip #view data 


,fips,name,state
0,0,UNITED STATES,NaN
1,1000,ALABAMA,NaN
2,1001,Autauga County,AL
3,1003,Baldwin County,AL
4,1005,Barbour County,AL
...,...,...,...
3190,56037,Sweetwater County,WY
3191,56039,Teton County,WY
3192,56041,Uinta County,WY
3193,56043,Washakie County,WY


write the raw fip data to disc to upload to the github folder

In [21]:
fip.to_csv('fip_codes.csv')

Next, we need to append the 'fip'data to the 'mask' data by 'fip code' so that we have the names that correspond with each county name in our mask data. 

To merge the data by fip code, the variable names for the fip codes in the 'fip' and 'mask' data need to match up. 

Currently, this is the variable name for the fip codes in each dataset: 

mask data:'COUNTYFP'
fip:'fips'


In [ ]:
mask= mask.rename(columns=str.lower) #step1: change column names in mask data to lowercase#

In [22]:
fip.columns = ['countyfp', 'name','state'] #step 2: in the fips data, change the 'fips' column to 'countyfp' 

In [ ]:
fip.columns #view column to make sure names have been changed 

Now that the variable names match we can use pandas 'merge' function to merge fip and mask by 'countyfp'

In [ ]:
mask_data=mask.merge(fip, how='left', on='countyfp')
    #how = merge format (left)
    #on = column or index to merge on ('countyfp')

mask_data #view merged data 

'mask_data' includes data from the entire US. Our group is only including counties from Washington, California, and Oregon, so we need to filter the data so the 'state' variable only includes 'states': WA, CA, OR

In [25]:
states = ['WA','CA','OR']

mask_CA_WA_OR= mask_data[mask_data.state.isin(states)] 
  
mask_CA_WA_OR #dataframe that only include CA,WA,and OR

mask_CA_WA_OR.isnull().sum() #Make sure there are no NAs in the mask_CA_WA_OR dataframe#



countyfp      0
never         0
rarely        0
sometimes     0
frequently    0
always        0
name          0
state         0
dtype: int64

Next, we need to calculate a composite mask score for each county:

score= (%NEVER0) + (%RARELY1) + (%SOMETIMES2) + (%FREQUENTLY3) + (%ALWAYS*4)


In [26]:

mask_CA_WA_OR['mask_score'] = mask_CA_WA_OR['never']*0 + mask_CA_WA_OR['rarely']*1 + mask_CA_WA_OR['sometimes']*2 + mask_CA_WA_OR['frequently']*3+ mask_CA_WA_OR['always']*4

mask_CA_WA_OR.mask_score #view mask_score variable


<ipython-input-26-d74f51d8eb19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mask_CA_WA_OR['mask_score'] = mask_CA_WA_OR['never']*0 + mask_CA_WA_OR['rarely']*1 + mask_CA_WA_OR['sometimes']*2 + mask_CA_WA_OR['frequently']*3+ mask_CA_WA_OR['always']*4


186     3.667
187     3.279
188     3.395
189     3.381
190     3.291
        ...  
2987    3.330
2988    3.287
2989    3.504
2990    3.553
2991    3.562
Name: mask_score, Length: 133, dtype: float64

Prepare the dataset to be merged with group members (e.g., make sure the variable names are the same and are formatted correctly: 


(1) rename 'county' and 'state' to 'County' and 'State'

In [27]:

mask_CA_WA_OR.columns = ['countyfp', 'never','rarely','sometimes','frequently','always', 'County','State','mask_score']

mask_CA_WA_OR.columns


Index(['countyfp', 'never', 'rarely', 'sometimes', 'frequently', 'always',
       'County', 'State', 'mask_score'],
      dtype='object')

(2) The variable 'County' currently includes "Name of county" + "county". We want to remove "county" and trailing spaces so that this varibale only includes the name of the county. 

Example: "Alameda County" == "Alameda"

In [28]:
mask_CA_WA_OR['County'] = mask_CA_WA_OR['County'].str.replace(' County', '')

<ipython-input-28-1cac62b79fda>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mask_CA_WA_OR['County'] = mask_CA_WA_OR['County'].str.replace(' County', '')


We don't need the fip code variable ('countyfp') so we can remove this variable:

In [29]:
del mask_CA_WA_OR['countyfp']

mask_CA_WA_OR #view data


,never,rarely,sometimes,frequently,always,County,State,mask_score
186,0.019,0.008,0.055,0.123,0.795,Alameda,CA,3.667
187,0.025,0.085,0.088,0.190,0.612,Alpine,CA,3.279
188,0.045,0.013,0.099,0.188,0.655,Amador,CA,3.395
189,0.015,0.043,0.111,0.204,0.626,Butte,CA,3.381
190,0.045,0.019,0.098,0.276,0.562,Calaveras,CA,3.291
...,...,...,...,...,...,...,...,...
2987,0.045,0.057,0.079,0.161,0.658,Wahkiakum,WA,3.330
2988,0.083,0.022,0.061,0.193,0.641,Walla Walla,WA,3.287
2989,0.042,0.028,0.061,0.122,0.747,Whatcom,WA,3.504
2990,0.002,0.009,0.049,0.310,0.629,Whitman,WA,3.553


There are some duplicate county names ((e.g., there is county 'x' in state 'z' and state 'y'). This will cause an error when we conduct the clustering analyses. To address this problem, we will create a 'Location' variable and concatenate the 'County' and 'State' variable

In [33]:
mask_CA_WA_OR['Location']= mask_CA_WA_OR['County'] +"_"+mask_CA_WA_OR['State']

mask_CA_WA_OR.Location#view variable


<ipython-input-33-8292d3cb41a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mask_CA_WA_OR['Location']= mask_CA_WA_OR['County'] +"_"+mask_CA_WA_OR['State']


186         Alameda_CA
187          Alpine_CA
188          Amador_CA
189           Butte_CA
190       Calaveras_CA
             ...      
2987      Wahkiakum_WA
2988    Walla Walla_WA
2989        Whatcom_WA
2990        Whitman_WA
2991         Yakima_WA
Name: Location, Length: 133, dtype: object

Remove 'State' and 'County' variables:

In [35]:
del mask_CA_WA_OR['State']
del mask_CA_WA_OR['County']

In [36]:
mask_CA_WA_OR #view dataframe

,never,rarely,sometimes,frequently,always,mask_score,Location
186,0.019,0.008,0.055,0.123,0.795,3.667,Alameda_CA
187,0.025,0.085,0.088,0.190,0.612,3.279,Alpine_CA
188,0.045,0.013,0.099,0.188,0.655,3.395,Amador_CA
189,0.015,0.043,0.111,0.204,0.626,3.381,Butte_CA
190,0.045,0.019,0.098,0.276,0.562,3.291,Calaveras_CA
...,...,...,...,...,...,...,...
2987,0.045,0.057,0.079,0.161,0.658,3.330,Wahkiakum_WA
2988,0.083,0.022,0.061,0.193,0.641,3.287,Walla Walla_WA
2989,0.042,0.028,0.061,0.122,0.747,3.504,Whatcom_WA
2990,0.002,0.009,0.049,0.310,0.629,3.553,Whitman_WA


In [37]:
#export to .csv#
mask_CA_WA_OR.to_csv('bryn.bl.clean.data.csv',index=False)
